**Ysrael Patrick do Nascimento Tavares 20182430002**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import lu, solve_triangular, solve
np.set_printoptions(formatter={'float': lambda x: "{:.4f}".format(x)}) # configura a impressão dos floats

# Projeto de Sistemas Lineares

![]()

Uma empresa de construção teve uma visão um tanto peculiar nos seus projetos, os espaços domesticos feitos pela empresa tem as seguintes particularidades:

* Todas as 4 paredes do espaço tem a mesmas dimensões(comprimento).
* A parede em frente a entrada tem uma janela, e tem a cor diferente das outras 3 paredes.
* O teto é revestido com gesso liso


Um estagiario foi contratado e sua primeira missão é calcular o valor total das tintas que serão usadas no proximo projeto.A

## O Problema

A ordem foi dada as pressas, não foi passado o contato do fornecedor onde são compradas as tintas. As unicas ferramentas que o estagiario tem são 3 orçamentos de projetos anteriores que usavam essas mesmas duas cores de tintas, onde só constam o valor total da compra e não o unitario.

Ao analisar os orçamentos ele tem os seguintes dados:

Parede, P | Área Comodo, A | Valor Total, T
----------|----------------|---------------
(P)       |      (A)       |      (T)
3x3       |      36        |     236.40
4x3       |      48        |     344.40
6x3       |      72        |     596.40

Sabendo que 1L de tinta rende $5m^2$ e obedecendo os critérios únicos da empresa que está participando ele faz algumas observações:

* A área do comodo é 4x a área de uma parede
* Uma tinta em apenas $1/4$ do comodo e a outra em $3/4$.
* Da área maior deve ser subtraida a área da porta(2.15x0.80 = $1,72m^2$)
* Da área menor deve ser subtraida a área da janela(1x1 = $1m^2$)
* Como o comodo é quadrado, a área do teto é igual a $Largura^2$

Depois de toda analise e com conhecimento de metodos númericos ele chega as seguintes funções:

$$f(L_1,A_1) = ((L_1A_1-1)/5)a+((3L_1A_1-1.72)/5)b + L_1^2c$$
$$f(L_2,A_2) = ((L_2A_2-1)/5)a+((3L_2A_2-1.72)/5)b + L_2^2c$$
$$f(L_3,A_3) = ((L_3A_3-1)/5)a+((3L_3A_3-1.72)/5)b + L_3^2c$$

onde $A_n$ é a área a ser calculada, $a$ e $b$ são os valores de cada tinta.

Após a substituição dos valores:

$$1.6a+5.056b+1.8c = 236.4$$
$$2.2a+6.856b+3.2c = 344.4$$
$$3.4a+10.456b+7.2c = 596.4$$

Logo ele percebe que para determinar os coeficientes do polinômio basta resolver o sistema acima.

## Resolvendo o sistema utilizando Gauss Ingênuo

Inicialmente ele decide resolver o sistema utilizando apenas o método de Gauss ingênuo para verificar se a sua premissa está correta.


In [ ]:
main = np.array([[1.6,5.056,1.8,236.4],[2.2,6.856,3.2,344.4],[3.4,10.456,7.2,596.4]])
main

array([[1.6000, 5.0560, 1.8000, 236.4000],
       [2.2000, 6.8560, 3.2000, 344.4000],
       [3.4000, 10.4560, 7.2000, 596.4000]])

In [ ]:
m = main.copy()
m

array([[1.6000, 5.0560, 1.8000, 236.4000],
       [2.2000, 6.8560, 3.2000, 344.4000],
       [3.4000, 10.4560, 7.2000, 596.4000]])

In [ ]:
pivo = m[0,0]

m1 = m[1,0]/pivo
m[1,:] -= m[0,:]*m1

m2 = m[2,0]/pivo
m[2,:] -= m[0,:]*m2

pivo = m[1,1]

m3 = m[2,1]/pivo
m[2,:] -= m[1,:]*m3


x = np.zeros(3)

x[2] = m[2,3]/m[2,2]
x[1] = (m[1,3] - m[1,2]*x[2])/m[1,1]
x[0] = (m[0,3] - m[0,2]*x[2] - m[0,1]*x[1])/m[0,0]
x

array([35.0000, 25.0000, 30.0000])

# Ele define uma função em Python para testar a sua aproximação polinomial

In [ ]:
def polinomio(l,a,x):
  a1 = (l*a-1)/5
  b = (3*l*a-1.72)/5
  c = l**2/5
  return np.around(a1*x[0]+b*x[1]+c*x[2],2)

Agora o seu supervisor pode saber o custo de tinta qualquer área entre $36m^2$ e $72m^2$. 

In [ ]:
for i in [[3,3],[4,3],[6,3]]:
  r = polinomio(i[0],i[1],x)
  print("O valor de custo de tinta para uma área de {}m^2  e gesso para um teto de {}m^2 é R${}".format(i[0]*i[1]*4,i[0]**2,r))

O valor de custo de tinta para uma área de 36m^2  e gesso para um teto de 9m^2 é R$236.4
O valor de custo de tinta para uma área de 48m^2  e gesso para um teto de 16m^2 é R$344.4
O valor de custo de tinta para uma área de 72m^2  e gesso para um teto de 36m^2 é R$596.4


## Resolvendo o sistema utilizando Gauss com pivotação parcial

Para verificar se é possível melhorar o seu resultado ele decide testar com outros métodos


In [ ]:
mp = main.copy()

In [ ]:
mp[[0,np.argmax(np.abs(mp[:,0]))]] = mp[[np.argmax(np.abs(mp[:,0])),0]]
mp

array([[3.4000, 10.4560, 7.2000, 596.4000],
       [2.2000, 6.8560, 3.2000, 344.4000],
       [1.6000, 5.0560, 1.8000, 236.4000]])

In [ ]:
pivo = mp[0,0]

m1 = mp[1,0]/pivo
mp[1,:] -= mp[0,:]*m1

m2 = mp[2,0]/pivo
mp[2,:] -= mp[0,:]*m2
mp

array([[3.4000, 10.4560, 7.2000, 596.4000],
       [0.0000, 0.0904, -1.4588, -41.5059],
       [0.0000, 0.1355, -1.5882, -44.2588]])

In [ ]:
mp[[1,2]] = mp[[2,1]]
mp

array([[3.4000, 10.4560, 7.2000, 596.4000],
       [0.0000, 0.1355, -1.5882, -44.2588],
       [0.0000, 0.0904, -1.4588, -41.5059]])

In [ ]:
pivo = mp[1,1]

m3 = mp[2,0]/pivo
mp[2,:] -= mp[1,:]*m1
mp

array([[3.4000, 10.4560, 7.2000, 596.4000],
       [0.0000, 0.1355, -1.5882, -44.2588],
       [0.0000, 0.0027, -0.4311, -12.8678]])

In [ ]:
xp = np.zeros(3)

xp[2] = mp[2,3]/mp[2,2]
xp[1] = (mp[1,3] - mp[1,2]*xp[2])/mp[1,1]
xp[0] = (mp[0,3] - mp[0,2]*xp[2] - mp[0,1]*xp[1])/mp[0,0]
xp

array([40.8796, 23.1942, 29.8459])

In [ ]:
print("Testando a resposta")
for i in [[3,3],[4,3],[6,3]]:
  r = polinomio(i[0],i[1],x)
  print("O valor de custo de tinta para uma área de {}m^2  e gesso para um teto de {}m^2 é R${}".format(i[0]*i[1]*4,i[0]**2,r))

Testando a resposta
O valor de custo de tinta para uma área de 36m^2  e gesso para um teto de 9m^2 é R$236.4
O valor de custo de tinta para uma área de 48m^2  e gesso para um teto de 16m^2 é R$344.4
O valor de custo de tinta para uma área de 72m^2  e gesso para um teto de 36m^2 é R$596.4


## Resolvendo o sistema utilizando Gauss-Seidel

$$1.6a+5.056b+1.8c = 236.4$$
$$2.2a+6.856b+3.2c = 344.4$$
$$3.4a+10.456b+7.2c = 596.4$$



In [ ]:
max_it = 1000
tolerancia = 0.001
xs = np.zeros(3)

for i in range(max_it):
  xs_anterior = xs.copy()
  xs[0] = (236.4 - 5.056*xs[1] - 1.8*xs[2])/1.6
  xs[1] = (344.4 - 2.2*xs[0] - 3.2*xs[2])/6+856
  xs[2] = (596.4 - 3.4*xs[0] - 10.456*xs[1])/7.2
  
  erros = np.abs(np.divide(xs - xs_anterior,xs))*100
  print(i, xs, np.max(erros))
  if np.max(erros) < tolerancia:
    break
  

0 [147.7500 859.2250 -1234.7231] 100.0
1 [-1178.3375 2003.9761 -2270.9481] 112.5388527411197
2 [-3629.9978 3455.5715 -3221.2588] 67.53889277636998
3 [-7147.9398 5252.3160 -4169.2806] 49.216167850178614
4 [-11759.1278 7448.6965 -5181.4078] 39.213690236042105
5 [-17561.0472 10115.8681 -6314.9384] 33.03857300669815
6 [-24714.0875 13343.1993 -7623.9158] 28.943169974682498
7 [-33439.8544 17240.7684 -9163.5513] 26.093913991627854
8 [-44024.0829 21942.7911 -10993.8252] 24.041905700494283
9 [-56823.4164 27612.0261 -13182.6847] 22.524751828248778
10 [-72275.7324 34445.2670 -15809.1419] 21.37967399313663
11 [-90914.0091 42680.0790 -18966.5105] 20.500995307403617
12 [-113383.9755 52602.9966 -22765.9744] 19.817585568990765
13 [-140465.9980 64559.4523 -27340.6722] 19.28012681478147
14 [-173101.8630 78965.7749 -32850.4734] 18.853560802757773
15 [-212427.3162 96323.6684 -39487.6392] 18.5124276518232
16 [-259811.4482 117237.6719 -47483.5797] 18.237892250070672
17 [-316904.2661 142436.2067 -57116.9545]

# Mais trabalho pro estagiário

Muito satisfeito com o trabalho que você apresentou o supervisor passa mais 2 projetos pra você, cada um com 3 relatorios referente as cores q serão usadas.

Após a analise dos relatórios você obtém dados abaixo:

## Conjunto 2:

Parede, P | Área Comodo, A | Valor Total, T
----------|----------------|---------------
(P)       |      (A)       |      (T)
3x3       |      36        |     289.56
4x3       |      48        |     414.36
6x3       |      72        |     696.36

## Conjunto 3:

Parede, P | Área Comodo, A | Valor Total, T
----------|----------------|---------------
(P)       |      (A)       |      (T)
3x3       |      36        |     297.27
4x3       |      48        |     421.87
6x3       |      72        |     698.67


# A Análise

Olhando os dados ele percebe que quando você escreve os sistemas de equações que a matriz de coeficientes é a mesma para todos os projetos, logo ele lembra dos ensinamentos da disciplina de Cálculo Numérico e decide resolver os sistemas utilizando **Decomposição LU** 


In [ ]:
A = np.array([[1.6,5.056,1.8],[2.2,6.856,3.2],[3.4,10.456,7.2]])
B = np.array([236.4,344.4,596.4])
l = np.eye(3,3)
A

array([[1.6000, 5.0560, 1.8000],
       [2.2000, 6.8560, 3.2000],
       [3.4000, 10.4560, 7.2000]])

In [ ]:
pivo = A[0,0]

m10 = A[1,0]/pivo
A[1,:] -= A[0,:]*m10
l[1,0]= m10

m20 = A[2,0]/pivo
A[2,:] -= A[0,:]*m20
l[2,0]=m20

pivo = A[1,1]

m21 = A[2,1]/pivo
A[2,:] -= A[1,:]*m21
l[2,1]=m21

l

array([[1.0000, 0.0000, 0.0000],
       [1.3750, 1.0000, 0.0000],
       [2.1250, 3.0000, 1.0000]])

In [ ]:
Z = np.zeros(A.shape[0])
Z = solve_triangular(l,B, lower=True)
Z

array([236.4000, 19.3500, 36.0000])

In [ ]:
xlu = solve_triangular(A,Z)
xlu

array([35.0000, 25.0000, 30.0000])

### Resolvendo para os dados do Projeto #2

* Obtendo [Z]

In [ ]:
B2 = np.array([289.56,414.36,696.36])
Z2 = solve_triangular(l,B2,lower=True)
Z2

array([289.5600, 16.2150, 32.4000])

* Obtendo [X]

In [ ]:
X2 = solve_triangular(A,Z2)
X2

array([40.0000, 35.0000, 27.0000])

In [ ]:
for i in [[3,3],[4,3],[6,3]]:
  r = polinomio(i[0],i[1],X2)
  print("O valor de custo de tinta para uma área de {}m^2  e gesso para um teto de {}m^2 é R${}".format(i[0]*i[1]*4,i[0]**2,r))

O valor de custo de tinta para uma área de 36m^2  e gesso para um teto de 9m^2 é R$289.56
O valor de custo de tinta para uma área de 48m^2  e gesso para um teto de 16m^2 é R$414.36
O valor de custo de tinta para uma área de 72m^2  e gesso para um teto de 36m^2 é R$696.36


### Resolvendo para os dados do projeto #3

* Obtendo [Z]

In [ ]:
B3 = np.array([297.27,421.87,698.67])
Z3 = solve_triangular(l,B3,lower=True)
Z3

array([297.2700, 13.1238, 27.6000])

* Obtendo [X]

In [ ]:
X3 = solve_triangular(A,Z3)
X3

array([43.0234, 36.9922, 23.0000])

In [ ]:
for i in [[3,3],[4,3],[6,3]]:
  r = polinomio(i[0],i[1],X3)
  print("O valor de custo de tinta para uma área de {}m^2  e gesso para um teto de {}m^2 é R${}".format(i[0]*i[1]*4,i[0]**2,r))

O valor de custo de tinta para uma área de 36m^2  e gesso para um teto de 9m^2 é R$297.27
O valor de custo de tinta para uma área de 48m^2  e gesso para um teto de 16m^2 é R$421.87
O valor de custo de tinta para uma área de 72m^2  e gesso para um teto de 36m^2 é R$698.67


# Conclusão

## Gauss Ingenuo

O resultado obtido foi exato devido a não ocorrer nenhuma divisão por zero no meio do algoritmo.

## Gauss com Pivotação

O metodo conseguiu resolver o sistema e dar o resultado total esperado, mas o valor de cada incognita foi diferente do real.

## Gauss-Seidel

O metodo não convergiu em 1000 iterações, resultado esperado, pois a matriz não possuia diagonal principal predominante.

## Decomposição LU

O metodo conseguiu resolver os 2 problemas adcionais encontrando as soluções esperadas.